In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Import libraries
from scipy import stats
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("white")
sns.set(style="ticks", color_codes=True)
%matplotlib inline
from sklearn.model_selection import learning_curve, validation_curve, cross_val_score

# Create table for missing data analysis
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

# missing data
def find_missing_data(df):
    #missing data
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    if missing_data['Total'].max() > 0: print (missing_data.head(20))
    else: print ("No missing data.")

**Load Data and Explore **





In [ ]:
# Input data files are available in the "../input/" directory.
bckgrnd = pd.read_excel('../input/UC1.xlsx', 'Background', index_col=None)
flwseeds = pd.read_excel('../input/UC1.xlsx', 'Data', index_col=None)
# bckgrnd.dropna(axis=0, how='all') #remve all row where all value is 'NaN' exists
# Any results will be writen to the current directory are saved as output.
flwseeds.head()

In [ ]:
flwseeds = flwseeds.drop(["Unnamed: 0"], axis=1)
flwseeds.info()

*Check for missing data*

In [ ]:
draw_missing_data_table(flwseeds) ## None

In [ ]:
flwseeds["STORE CODE"].unique().size ## all unique store codes

*Distribution Plots*

In [ ]:
initStats = flwseeds.describe()
initStats = initStats.drop(columns =["STORE CODE"])
initStats

In [ ]:
flw_names = list(flwseeds.columns.values)[1:]

initStats = pd.DataFrame(columns=["mean","std"])
initStats["mean"]=flwseeds.drop(columns =["STORE CODE"]).mean();
initStats["std"]=flwseeds.drop(columns =["STORE CODE"]).std();
initStats["count"]=flwseeds.drop(columns =["STORE CODE"]).sum();
initStats["Name"]=flw_names
# sort df by Count column
initStats = initStats.sort_values(['mean'], ascending=False).reset_index(drop=True)

f, ax = plt.subplots(figsize=(8, 18))
sns.barplot(initStats["mean"],initStats.Name,palette="Blues_d")

# print ("Most uncommon seeds: \n",initStats.tail(20))
initStats.tail(20)

Everybody orders Alyssum Y. and more than 95% order Calendula Gold, Marigold Jafri Black, Anthurium Mix, Cleome Rose, Linum Blue, Holly Hock, Sweet Pea Mix, Cereopsis, Sweet Pea Pink, Sweet Pea White and Delphinium. The rest of the flowers on the list would benefit most from sale recommendations.

*Correlation Matrix*

In [ ]:
flw_train = flwseeds.loc[:,flw_names]
corrmat = flw_train.corr(method='pearson')
f, ax = plt.subplots(figsize=(18, 18))
sns.heatmap(corrmat, square=True)#,annot=True);

1. Recommendations based on **Pearsons' R** values

In [ ]:
print(initStats.loc[initStats['Name'] == "Marigold Jafri Black"]["mean"].values[0] )
def recommend(FlowerName, min_count):
    print("For flower seed ({})".format(FlowerName))
    print("- Top 10 seed recommendation based on Pearsons'R correlation - ")

    target = flw_train[FlowerName]
    similar_to_target = flw_train.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns = ['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    popularity = []
    for idx in corr_target.index.values:
        popularity.append(initStats.loc[initStats["Name"] == idx]["mean"].values[0] *100.)
    corr_target['Popularity (Seed Sale %)'] = popularity
    print(corr_target.head(10))    
    

In [ ]:
recommend("Calendula Gold", 10) #popular seed

In [ ]:
recommend("Petunia Purple", 10) # uncommon seed

To make seed recommendations: 

2. SVD
SVD handles the problem of scalability and sparsity posed by CF successfully. However, SVD is not without flaw. The main drawback of SVD is that there is no to little explanation to the reason that we recommend an item to an user. This can be a huge problem if users are eager to know why a specific item is recommended to them.

**Edelweiss wants this recommendation to be statistically correct. They also want to see if this data is meaningful enough to generate some decent recommendation or not. If Kayla says that this data is insufficient, then she needs to validate that with some statistical results. They also would want to see some stores and some seeds where they should focus primarily. Other than this analysis, if the Edelweiss CEO likes the approach, then he would want to do a pilot of a few stores. Kayla should help the CEO to selct store list along with the recommnedations.**